In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = [15, 8]

import seaborn as sns
sns.set(style="ticks")

In [2]:
# Directorio donde se encuentran los datos.
DATA_RAW_PATH = "../../data/raw/nooa/spain"
DATA_PROCESSED_PATH = "../../data/processed/nooa/spain"

# Nombre de los ficheros CSV de entrada.
FILE_ESTACIONES = "estaciones.csv.xz"
FILE_DATOS = "datos.csv.xz"

In [3]:
datos_columns = [
    "id_estacion",
    "fecha",
    "tmax",
    "tmin",
    "precip",
    "nevada",
    "prof_nieve"
]

# Lectura y carga de los datos a un DataFrame pandas.
datos = pd.read_csv("{}/{}".format(DATA_RAW_PATH, FILE_DATOS),
                    sep = ";",
                    header=None,
                    names = datos_columns,
                    index_col = [0, 1],
                    parse_dates = ["fecha"],
                    infer_datetime_format = True,
                    compression="xz")

print(f"Dimensiones: {datos.shape}")

Dimensiones: (2953942, 5)


In [4]:
estaciones_columns = [
    "id_estacion",
    "longitud",
    "latitud",
    "altitud",
    "dummy01",
    "nombre",
    "gsn",
    "dummy02",
    "dummy03",
    "inicio",
    "fin"
]

# Lectura y carga de los datos a un DataFrame pandas.
estaciones = pd.read_csv("{}/{}".format(DATA_RAW_PATH, FILE_ESTACIONES),
                         sep = ";",
                         header=None,
                         names = estaciones_columns,
                         index_col = 0,
                         compression="xz")

print(f"Dimensiones: {estaciones.shape}")

Dimensiones: (207, 10)


In [5]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2953942 entries, ('SP000003195', Timestamp('1920-01-01 00:00:00')) to ('SPW00014011', Timestamp('2005-12-17 00:00:00'))
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   tmax        float64
 1   tmin        float64
 2   precip      float64
 3   nevada      float64
 4   prof_nieve  float64
dtypes: float64(5)
memory usage: 130.9+ MB


In [6]:
datos.describe()

,tmax,tmin,precip,nevada,prof_nieve
count,2.821753e+06,2.808019e+06,2.861705e+06,25898.000000,33005.000000
mean,2.002093e+02,9.905756e+01,1.683505e+01,0.033516,42.877594
std,7.851216e+01,6.797491e+01,5.978685e+01,1.383056,167.442243
min,-1.960000e+02,-9.990000e+02,0.000000e+00,0.000000,0.000000
25%,1.450000e+02,5.000000e+01,0.000000e+00,0.000000,0.000000
50%,2.000000e+02,1.000000e+02,0.000000e+00,0.000000,0.000000
75%,2.570000e+02,1.500000e+02,1.000000e+00,0.000000,0.000000
max,4.720000e+02,3.320000e+02,3.600000e+03,119.000000,2499.000000


In [7]:
estaciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 207 entries, SP000003195 to SPW00014011
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   longitud  207 non-null    float64
 1   latitud   207 non-null    float64
 2   altitud   207 non-null    float64
 3   dummy01   0 non-null      float64
 4   nombre    207 non-null    object 
 5   gsn       9 non-null      object 
 6   dummy02   0 non-null      float64
 7   dummy03   84 non-null     float64
 8   inicio    207 non-null    int64  
 9   fin       207 non-null    int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 17.8+ KB


In [8]:
estaciones.describe()

,longitud,latitud,altitud,dummy01,dummy02,dummy03,inicio,fin
count,207.000000,207.000000,207.000000,0.0,0.0,84.000000,207.000000,207.000000
mean,40.405733,-1.738512,494.638647,NaN,NaN,15009.880952,1975.144928,2015.951691
std,3.155343,4.444118,585.856829,NaN,NaN,17599.599587,32.122162,13.123298
min,27.818900,-17.888900,1.000000,NaN,NaN,8001.000000,1896.000000,1947.000000
25%,39.917200,-4.016800,60.000000,NaN,NaN,8118.750000,1947.000000,2020.000000
50%,41.478900,0.326400,287.000000,NaN,NaN,8229.000000,1973.000000,2020.000000
75%,42.189750,1.516250,690.500000,NaN,NaN,8417.500000,2008.000000,2021.000000
max,43.566700,4.215600,2535.000000,NaN,NaN,60338.000000,2017.000000,2021.000000


# Preprocesamiento

* tmin: Valores inválidos a -999 los pongo a NaN. Luego se procesarán.
* Columnas de nieve y precipitación:
  * Pongo a cero los valores nulos.
  * En el caso de nieve practicamente el 99% son nulos pero, aún así, no elimino las variables ya que son situaciones excepcionales que deben estar incluidas.
* Finalmente, elimino las filas donde alguno de los campos de temperatura sea nulo.

In [9]:
def show_missing_percentage(data, filter):
    col_number = 0
    pct_missing = {}
    for col in data.columns:
        pct_missing[col] = np.mean(data[col].isnull())
    for col in sorted(pct_missing, key=pct_missing.__getitem__, reverse=True):
        if pct_missing[col] > filter:
            col_number += 1
            print('{: <30}: {}%'.format(col, round(pct_missing[col]*100, 2)))
    if col_number == 0:
        print("Ninguna variable tiene valores perdidos por encima del {}%".format(filter*100))
            
show_missing_percentage(datos, 0)

nevada                        : 99.12%
prof_nieve                    : 98.88%
tmin                          : 4.94%
tmax                          : 4.48%
precip                        : 3.12%


In [10]:
datos.loc[datos["tmin"] == -999, "tmin"] = None

In [11]:
columns = ["precip", "nevada", "prof_nieve"]
for column in columns:
    datos.loc[datos[column].isna(), column] = 0

In [12]:
datos.dropna(how="any", inplace=True)

In [13]:
show_missing_percentage(datos, 0)

Ninguna variable tiene valores perdidos por encima del 0%


In [14]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2778702 entries, ('SP000003195', Timestamp('1920-01-01 00:00:00')) to ('SPW00014011', Timestamp('1967-12-31 00:00:00'))
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   tmax        float64
 1   tmin        float64
 2   precip      float64
 3   nevada      float64
 4   prof_nieve  float64
dtypes: float64(5)
memory usage: 123.3+ MB


In [15]:
datos.describe()

,tmax,tmin,precip,nevada,prof_nieve
count,2.778702e+06,2.778702e+06,2.778702e+06,2.778702e+06,2.778702e+06
mean,2.001745e+02,9.876497e+01,1.643283e+01,2.929425e-04,4.788466e-01
std,7.857347e+01,6.783950e+01,5.910200e+01,1.299941e-01,1.830345e+01
min,-1.960000e+02,-3.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.450000e+02,5.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+02,1.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.570000e+02,1.500000e+02,1.000000e+00,0.000000e+00,0.000000e+00
max,4.720000e+02,3.320000e+02,3.600000e+03,1.190000e+02,2.499000e+03


# Transformación

* Generar medias semanales.
* Elimina observaciones con nulos, es decir, semanas donde no hay datos registrados.
* Agrega geolocalización.
* Crea nueva columna `semana`.

In [16]:
# Agrupo por estación (level=0) y calculo la media mensual por el campo fecha (level=1).
datos_semana = datos.groupby(level=0).resample("W", level=1).mean().round(decimals=0)

In [17]:
show_missing_percentage(datos_semana, 0)

tmax                          : 5.96%
tmin                          : 5.96%
precip                        : 5.96%
nevada                        : 5.96%
prof_nieve                    : 5.96%


In [18]:
# Elimina nulos.
datos_semana.dropna(how="all", inplace=True)

In [19]:
show_missing_percentage(datos_semana, 0)

Ninguna variable tiene valores perdidos por encima del 0%


In [20]:
# Genera nueva columna `mes`.
datos_semana.reset_index(inplace=True)
datos_semana.insert(2, "fecha_cnt", datos_semana["fecha"].dt.isocalendar().week)

In [21]:
# Agrega datos geolocalización.
datos_geo = datos_semana.join(
    estaciones[["longitud", "latitud", "altitud"]],
    how="left",
    on="id_estacion"
)

In [22]:
datos_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408663 entries, 0 to 408662
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   id_estacion  408663 non-null  object        
 1   fecha        408663 non-null  datetime64[ns]
 2   fecha_cnt    408663 non-null  UInt32        
 3   tmax         408663 non-null  float64       
 4   tmin         408663 non-null  float64       
 5   precip       408663 non-null  float64       
 6   nevada       408663 non-null  float64       
 7   prof_nieve   408663 non-null  float64       
 8   longitud     408663 non-null  float64       
 9   latitud      408663 non-null  float64       
 10  altitud      408663 non-null  float64       
dtypes: UInt32(1), datetime64[ns](1), float64(8), object(1)
memory usage: 33.1+ MB


In [23]:
datos_geo

,id_estacion,fecha,fecha_cnt,tmax,tmin,precip,nevada,prof_nieve,longitud,latitud,altitud
0,SP000003195,1920-01-04,1,96.0,41.0,4.0,0.0,0.0,40.4117,-3.6781,667.0
1,SP000003195,1920-01-11,2,81.0,5.0,0.0,0.0,0.0,40.4117,-3.6781,667.0
2,SP000003195,1920-01-18,3,117.0,21.0,0.0,0.0,0.0,40.4117,-3.6781,667.0
3,SP000003195,1920-01-25,4,118.0,19.0,0.0,0.0,0.0,40.4117,-3.6781,667.0
4,SP000003195,1920-02-01,5,106.0,31.0,2.0,0.0,0.0,40.4117,-3.6781,667.0
...,...,...,...,...,...,...,...,...,...,...,...
408658,SPW00014011,1967-12-03,48,126.0,22.0,1.0,0.0,0.0,40.4833,-3.4500,608.1
408659,SPW00014011,1967-12-10,49,87.0,-13.0,0.0,0.0,0.0,40.4833,-3.4500,608.1
408660,SPW00014011,1967-12-17,50,64.0,-60.0,0.0,0.0,0.0,40.4833,-3.4500,608.1
408661,SPW00014011,1967-12-24,51,64.0,-16.0,2.0,0.0,0.0,40.4833,-3.4500,608.1


In [24]:
show_missing_percentage(datos_geo, 0)

Ninguna variable tiene valores perdidos por encima del 0%


# Dataframe a disco

In [25]:
datos_geo.to_csv("{}/{}".format(DATA_PROCESSED_PATH, "datos_semana.csv.xz"),
                 index=False,
                 sep = ";")